For introduction and problem statement, please refer to notebook 1

Note: this notebook may take more than 10 hours to run. To shorten the run time, I ran this notebook using Google Colab GPU.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
#google colab requires a pip install of transformers everytime its refreshed. hence this cell 
#is required if this notebook is to be run again on google colab
pip install transformers

     |████████████████████████████████| 675kB 7.1MB/s 
     |████████████████████████████████| 3.8MB 18.4MB/s 
     |████████████████████████████████| 890kB 41.5MB/s 
     |████████████████████████████████| 1.1MB 42.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=93927d15df1703dda79bc8072aa28c963bf570677f8962742aa633b16b9b2db1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
#importing the library
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import spacy
from nltk import tokenize
from nltk.corpus import stopwords 
import re
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

In [4]:
#importing the csv file
new_reviews = pd.read_csv('/content/drive/My Drive/Capstone pytorch/cleaned_combined_data_with_keywords.csv',na_filter=False)

## With BERT

In [6]:
#importing libary
import torch
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [7]:
#confirm the shape
new_reviews.shape

(22041, 26)

In [9]:
new_reviews['multi_class_sentiment'].value_counts(normalize=True)
#baseline accuracy is 63.3%. It has also been shown when we did logistic
#regression modelling 

2    0.633592
0    0.269906
1    0.096502
Name: multi_class_sentiment, dtype: float64

In [10]:
#create train and validation set 
X_train, X_val, y_train, y_val = train_test_split(new_reviews.index.values,
                                                  new_reviews.multi_class_sentiment.values,
                                                  test_size = 0.25,
                                                  random_state= 42,
                                                  stratify=new_reviews.multi_class_sentiment.values)

In [11]:
#confirm the shape of train dataset
y_train.shape

(16530,)

In [12]:
#confirm the shape of validation dataset
y_val.shape

(5511,)

In [13]:
#create a data_type column that contains "not_set" as a placeholder before
# we label it with train and validation
new_reviews['data_type'] = ['not_set']*new_reviews.shape[0]

In [14]:
#label the rows with train or val
new_reviews.loc[X_train, 'data_type'] = 'train'
new_reviews.loc[X_val, 'data_type'] = 'val'

In [15]:
new_reviews.head(2)

,asin,name,rating,date,verified,review_title,body,helpfulVotes,brand,item_title,url,image,reviewUrl,totalReviews,price,originalPrice,sellers,reviews,cleaned_reviews,pos_neg_reviews,tokens,summary,features_and_sentiments,filter summary,data_type,multi_class_sentiment
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0,,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.0,0.0,Flip n Smart Phones,"Def not best, but not worst I had the Samsung ...",def best worst samsung awhile absolute doo doo...,0,"['def', 'best', 'worst', 'samsung', 'awhile', ...","['nice bright large screen', 'ring tone loud e...","[(5, 'screen'), (5, 'battery')]",True,val,1
1,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0,,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.0,0.0,Flip n Smart Phones,"Love This Phone This is a great, reliable phon...",love great reliable also purchased samsung die...,1,"['love', 'great', 'reliable', 'also', 'purchas...",['however ringtones not available online downl...,"[(3, 'ringtones')]",True,train,2


In [16]:
#sanity check to ensure that the train and val are 
#represented correctly and evenly within each class
new_reviews.groupby(['multi_class_sentiment', 'data_type']).count()

asin  ...  filter summary
multi_class_sentiment data_type         ...                
0                     train       4462  ...            4462
                      val         1487  ...            1487
1                     train       1595  ...            1595
                      val          532  ...             532
2                     train      10473  ...           10473
                      val         3492  ...            3492

[6 rows x 24 columns]

## Loading Tokenizer and

In [18]:
#importing libraries
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [19]:
#loading BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [21]:
#prepare the data in a format that is readable by BERT
encoded_data_train = tokenizer.batch_encode_plus(
    new_reviews[new_reviews.data_type=='train'].reviews.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    new_reviews[new_reviews.data_type=='val'].reviews.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(new_reviews[new_reviews.data_type=='train'].multi_class_sentiment.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val= torch.tensor(new_reviews[new_reviews.data_type=='val'].multi_class_sentiment.values)

In [22]:
#preparing the data for BERT input
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [24]:
#sanity check
len(dataset_val)

5511

In [24]:
#sanity check
len(dataset_val)

5511

## Setting up BERT Pretrained Model

In [25]:
#import library
from transformers import BertForSequenceClassification

In [26]:
#instantiate the pre-trained model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)


## Creating Data Loaders

In [27]:
#importing library
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [28]:
#defining batchsize
batch_size = 32

#final processing of data input
dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [29]:
dataloader_train 

In [30]:
16530/32

516.5625

## Setting Up Optimiser and Scheduler

In [31]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [32]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [33]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

## Creating Training Loop

In [35]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [37]:
def evaluate(dataloader_val):
    """
    to evaluate training loss while running the model - will be used in 
    this notebook to choose the best epoch
    
    Parameter
    ----------
    dataloader_val: the validation set that has been processed 

    Returns
    -------
    validation loss, predictions of target variable and actual target variable values 
    """
    #activate model evaluation
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    #loop through the validationd data set by batch
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        #index the batch to find input_ids,attention_masks and labels
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }
        #disable gradient calculation
        with torch.no_grad():        
            outputs = model(**inputs)
        #index the outputs to find loss and logits    
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        
        #convert tensor to numpy array
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [38]:
#wrapping every epoch with tqdm to show the progress
for epoch in tqdm(range(1, epochs+1)):
    #setting the model to training mode
    model.train()
    
    loss_train_total = 0
    
    #wrapping the dataloader input in tqdm to create progress bar 
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    
    #fine-tuning the model
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    #save the model and export it     
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
    
    #print the epoch
    tqdm.write(f'\nEpoch {epoch}')
    
    #calculate training loss and print it as fine-tuning happens
    loss_train_avg = loss_train_total/len(dataloader_train)            
    #print the training loss
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    #calculate validation loss and print it as fine-tuning happens
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    
    tqdm.write(f'Validation loss: {val_loss}')
    


Epoch 1
Training loss: 0.47671686403986563



Validation loss: 0.3717515245079994



Epoch 2
Training loss: 0.3232948561442198



Validation loss: 0.3608986152432902



Epoch 3
Training loss: 0.2729301833884513



Validation loss: 0.364634551043283

